## Fixes:
- sort in absolute value
- Put it in a module / pip / conda
- Add differential expression feature
- Calculate everything in matrices rather than for loops
- When you color cells, what should the max and min expression be? Based on the selected cells or based on all of the cells. Currently it seems to remap based on only the selected cells but this is maybe misleading.

In [1]:
import pandas as pd
import numpy as np
from umap_explorer import UMAPexplorer


In [2]:
%load_ext py5
%gui osx

In [4]:
print('DOWNLOADING AND EXTRACTING EXAMPLE DATA')
! mkdir -p ../data
! wget https://storage.googleapis.com/sabeti-public/dkotliar/scnavigator/pbmc3k/data/pbmc3k_umap.tsv -O ../data/pbmc3k_umap.tsv
! wget https://storage.googleapis.com/sabeti-public/dkotliar/scnavigator/pbmc3k/data/pbmc3k_expression_filtered_normalized.tsv.gz -O ../data/pbmc3k_expression_filtered_normalized.tsv.gz
! gzip -df ../data/pbmc3k_expression_filtered_normalized.tsv.gz
! ls ../data

DOWNLOADING AND EXTRACTING EXAMPLE DATA
--2021-02-27 14:25:08--  https://storage.googleapis.com/sabeti-public/dkotliar/scnavigator/pbmc3k/data/pbmc3k_umap.tsv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.11.48, 172.217.7.16, 142.250.64.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.11.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143969 (141K) [text/tab-separated-values]
Saving to: ‘../data/pbmc3k_umap.tsv’

../data/pbmc3k_umap 100%[===================>] 140.59K  --.-KB/s    in 0.07s   

2021-02-27 14:25:08 (1.94 MB/s) - ‘../data/pbmc3k_umap.tsv’ saved [143969/143969]

--2021-02-27 14:25:08--  https://storage.googleapis.com/sabeti-public/dkotliar/scnavigator/pbmc3k/data/pbmc3k_expression_filtered_normalized.tsv.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.11.48, 172.217.7.16, 142.250.64.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.11.48|:443.

In [3]:
print("LOADING UMAP DATA...")

_umap = pd.read_csv('../data/pbmc3k_umap.tsv', sep='\t')
_umap.index = _umap['index']
_umap = _umap.iloc[:, 1:]
_umap.head()

LOADING UMAP DATA...


,UMAP_1,UMAP_2
index,,
AAACATACAACCAC-1,3.638991,3.122167
AAACATTGAGCTAC-1,-0.490064,10.553852
AAACATTGATCAGC-1,0.391078,4.035561
AAACCGTGCTTCCG-1,-6.747729,-0.600776
AAACCGTGTATGCG-1,1.887045,-1.526383


In [4]:
print("LOADING GENE EXPRESSION DATA...")

_expr = pd.read_csv('../data/pbmc3k_expression_filtered_normalized.tsv', sep='\t')
_expr.set_index('index', inplace=True)
_expr.head()

LOADING GENE EXPRESSION DATA...


,TNFRSF4,CPSF3L,ATAD3C,C1orf86,RER1,TNFRSF25,TNFRSF9,CTNNBIP1,SRM,UBIAD1,...,DSCR3,BRWD1,BACE2,SIK1,C21orf33,ICOSLG,SUMO3,SLC19A1,S100B,PRMT2
index,,,,,,,,,,,,,,,,,,,,,
AAACATACAACCAC-1,-0.171470,-0.280812,-0.046677,-0.475169,-0.544024,4.928496,-0.038028,-0.280573,-0.341788,-0.195361,...,-0.226570,-0.236269,-0.102943,-0.222116,-0.312401,-0.121678,-0.521229,-0.098269,-0.209095,-0.531203
AAACATTGAGCTAC-1,-0.214582,-0.372653,-0.054804,-0.683391,0.633951,-0.334837,-0.045589,-0.498264,-0.541914,-0.209017,...,-0.317531,2.568868,0.007155,-0.445372,1.629285,-0.058662,-0.857163,-0.266844,-0.313146,-0.596654
AAACATTGATCAGC-1,-0.376888,-0.295085,-0.057527,-0.520972,1.332648,-0.309362,-0.103108,-0.272526,-0.500799,-0.220228,...,-0.302938,-0.239801,-0.071774,-0.297857,-0.410920,-0.070431,-0.590720,-0.158656,-0.170876,1.379000
AAACCGTGCTTCCG-1,-0.285241,-0.281735,-0.052227,-0.484929,1.572680,-0.271825,-0.074552,-0.258875,-0.416752,-0.208471,...,-0.262978,-0.231807,-0.093818,-0.247770,2.552079,-0.097402,1.631684,-0.119462,-0.179120,-0.505669
AAACCGTGTATGCG-1,-0.256484,-0.220394,-0.046800,-0.345859,-0.333409,-0.208122,-0.069514,5.806440,-0.283112,-0.199355,...,-0.202237,-0.176765,-0.167350,-0.098665,-0.275836,-0.139482,-0.310095,-0.006877,-0.109614,-0.461946


In [9]:
test = UMAPexplorer(_umap, _expr)
test.explore_data()

Selected 341 cells
Calculating correlations...
Done
Selected gene FPR1
Min/max expression level for gene FPR1 -1.188632607460022 2.2649850845336914


In [6]:
test.selected_gene_name

'FCGR3A'

In [7]:
test.significant_genes

,R,P
FCGR3A,-0.764338,1.910272e-120
CD79B,-0.517137,6.686502e-44
LINC00494,-0.445119,1.196999e-31
AP001462.6,-0.445064,1.220061e-31
LY6G6F,-0.445062,1.220861e-31
...,...,...
FCN1,0.546087,1.031735e-49
GSTP1,0.581930,9.650422e-58
GPX1,0.630139,2.995342e-70
LGALS2,0.645518,1.012081e-74


In [8]:
test.selected_cells

,index,proj
0,AAACCGTGCTTCCG-1,0.442832
1,AAACGCTGTTTCTG-1,0.285313
2,AAAGAGACGCGAGA-1,0.760472
3,AAAGCAGATATCGG-1,0.646594
4,AAAGTTTGTAGCGT-1,0.935591
...,...,...
618,TTTAGCTGGATACC-1,0.693416
619,TTTAGGCTCCTTTA-1,0.955086
620,TTTATCCTGTTGTG-1,0.422584
621,TTTCAGTGTCTATC-1,0.837187


In [ ]:
FCGR3A, CD79B, LINC00494    ... GPX1, LGALS2, S100A8